<a href="https://colab.research.google.com/github/YagyanshB/SemEval-Task6-CS408/blob/main/Additional_Baseline_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Additional Baseline Techniques Implemented:

---

Apart from the **Baseline Techniques** of **Logistic Regression** and **Convolutional Neural Network**; we have alse executed a **Support Vector Machine (SVM)** model.

Parallely on further research, out of curiosity we had also implemented a **Random Forrest Classifier** as a **Baseline Technique**. The results from the Random Forrest Classifier is just for our own knowledge and learning purposes.




In [390]:
# Importing the required libraries and modules for our Machine Learning Model

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns
import re
import matplotlib.pyplot as plt
import missingno as ms
% matplotlib inline

import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [391]:
# Uploading the dataset we would be using for training and testing our model.
# It is essential that we upload the datasets on our Google Drive and specify
# the right file path.

from google.colab import drive
drive.mount('/content/drive')

train_file = 'drive/My Drive/olid-training-v1.0.tsv'

test_file_a = 'drive/My Drive/testset-levela.tsv' 
test_labels_a = 'drive/My Drive/labels-levela.csv' 

test_labels_b = 'drive/My Drive/labels-levelb.csv' 
test_file_b = 'drive/My Drive/testset-levelb.tsv' 

test_file_c = 'drive/My Drive/testset-levelc.tsv' 
test_labels_c = 'drive/My Drive/labels-levelc.csv' 

# Within the Code below, we re run the code to ensure that our files have been
# mounted in the right manner. It is imperative to have the link to the directory
# sent correctly; else the files won't be loaded.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [392]:
training_data = pd.read_csv('drive/My Drive/olid-training-v1.0.tsv', sep='\t') #to read and store in panda dataframe

testing_data_a = pd.read_csv('drive/My Drive/testset-levela.csv') #to read and store in panda dataframe for Subtask A

testing_data_b = pd.read_csv('drive/My Drive/testset-levelb.csv') #to read and store in panda dataframe for Subtask B

testing_data_c = pd.read_csv('drive/My Drive/testset-levelc.csv') #to read and store in panda dataframe for Subtask C


In [393]:
training_data.shape, testing_data_a.shape  #to print number of rows and columns

((13240, 5), (860, 2))

In [394]:
training_data.shape, testing_data_b.shape  #to print number of rows and columns

((13240, 5), (240, 2))

In [395]:
training_data.shape, testing_data_c.shape  #to print number of rows and columns

((13240, 5), (213, 2))

In [396]:
training_data.head(10) #to print top 10 tweets with labels from training data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN
7,52415,@USER was literally just talking about this lo...,OFF,TIN,GRP
8,45157,@USER Buy more icecream!!!,NOT,NaN,NaN
9,13384,@USER Canada doesn’t need another CUCK! We alr...,OFF,TIN,IND


In [397]:
training_data.info() #complete information of the training data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         13240 non-null  int64 
 1   tweet      13240 non-null  object
 2   subtask_a  13240 non-null  object
 3   subtask_b  4400 non-null   object
 4   subtask_c  3876 non-null   object
dtypes: int64(1), object(4)
memory usage: 517.3+ KB


In [398]:
testing_data_a.head(10)  #to print top 10 rows testing data

,id,tweet
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1,27014,"#ConstitutionDay is revered by Conservatives, ..."
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
3,13876,#Watching #Boomer getting the news that she is...
4,60133,#NoPasaran: Unity demo to oppose the far-right...
5,83681,. . . What the fuck did he do this time?
6,96874,#RAP is a form of ART! Used to express yoursel...
7,65507,@USER Do you get the feeling he is kissing @US...
8,78910,5 Tips to Enhance Audience Connection on Faceb...
9,46363,#BiggBossTamil janani won the task. She is goi...


In [399]:
#function to drop unwanted features

def drop_features(features,data):
    data.drop(features,inplace=True,axis=1)

In [400]:
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

In [401]:
training_data['processed_tweets'] = training_data['tweet'].apply(process_tweet) 

# This processes our tweets and we can notice the difference between our orignal tweet
# and cleaned tweets provided in the table below.

In [402]:
training_data.head(15)

,id,tweet,subtask_a,subtask_b,subtask_c,processed_tweets
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,she should ask a few native americans what the...
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,go home you re drunk maga trump2020 url
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,amazon is investigating chinese employees who ...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,someone should vetaken this piece of shit to a...
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,obama wanted liberals amp illegals to move int...
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH,liberals are all kookoo
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN,oh noes tough shit
7,52415,@USER was literally just talking about this lo...,OFF,TIN,GRP,was literally just talking about this lol all ...
8,45157,@USER Buy more icecream!!!,NOT,NaN,NaN,buy more icecream
9,13384,@USER Canada doesn’t need another CUCK! We alr...,OFF,TIN,IND,canada doesn t need another cuck we already ha...


In [403]:
drop_features(['id','tweet'],training_data)

In [404]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   subtask_a         13240 non-null  object
 1   subtask_b         4400 non-null   object
 2   subtask_c         3876 non-null   object
 3   processed_tweets  13240 non-null  object
dtypes: object(4)
memory usage: 413.9+ KB


In [405]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(training_data["processed_tweets"], training_data["subtask_a"], test_size = 0.2, random_state = 42)

In [406]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [407]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [408]:
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(10592, 16528)
(10592, 16528)


In [409]:
x_train_counts

<10592x16528 sparse matrix of type '<class 'numpy.int64'>'
	with 96990 stored elements in Compressed Sparse Row format>

In [410]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [411]:
print(x_test_counts.shape)
print(x_test_tfidf.shape)

(2648, 16528)
(2648, 16528)


In [412]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [413]:
predictions = model.predict(x_test_tfidf)

In [414]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

array([[1614,  119],
       [ 518,  397]])

In [415]:
rf_f1=f1_score(y_test,predictions, pos_label='NOT')
rf_f1

0.8351875808538163

In [416]:
rf_f2=f1_score(y_test,predictions, pos_label='OFF')
rf_f2

0.5548567435359888

In [417]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(x_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [418]:
predict_svm = lin_clf.predict(x_test_tfidf)

In [419]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predict_svm)

array([[1518,  215],
       [ 438,  477]])

In [420]:
svm_f1=f1_score(y_test,predict_svm, pos_label='OFF')
svm_f1

0.5936527691350342

In [421]:
svm_f2=f1_score(y_test,predict_svm, pos_label='NOT')
svm_f2

0.822987259419897

In [428]:
results = { 'SVM':svm_f2}  
df = pd.DataFrame(results, index =['f1Score']) 
df

,SVM
f1Score,0.822987


In [423]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}
from sklearn.metrics import classification_report
clf = GridSearchCV(text_clf, tuned_parameters, cv=10)
clf.fit(x_train, y_train)

print(classification_report(y_test, clf.predict(x_test), digits=4))

              precision    recall  f1-score   support

         NOT     0.7080    0.9527    0.8123      1733
         OFF     0.7405    0.2557    0.3802       915

    accuracy                         0.7119      2648
   macro avg     0.7242    0.6042    0.5962      2648
weighted avg     0.7192    0.7119    0.6630      2648

